# ResNet18

In [20]:
from barbar import Bar
import copy
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
from torchvision import datasets, models, transforms
from tqdm import tqdm
import utils

## 1. Model construction

In [21]:
resnet18 = models.resnet18(pretrained=True).to(utils.device)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, utils.num_classes, device = utils.device)

In [22]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [23]:
# training parameters
LEARNING_RATE = 0.001
EPOCHS = 4
BATCH_SIZE = 128

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 2 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.1)

## 2. Split train / validation

In [24]:
# load all train videos (labelled videos)
all_train_videos = utils.get_train_test_video_names()['train']
all_train_labels = pd.read_pickle(utils.labels_path)

# define split
split = 0.8
np.random.seed(69)
train_videos = np.array(all_train_videos)[np.random.choice(len(all_train_videos), int(0.8 * len(all_train_videos)), replace=False)]
validation_videos = np.setdiff1d(all_train_videos, train_videos, assume_unique=False)
train_videos.sort()
validation_videos.sort()

In [25]:
# create two subdataframes for training and validation
training_df = all_train_labels.loc[all_train_labels['videoname'].isin(train_videos)]
validation_df = all_train_labels.loc[all_train_labels['videoname'].isin(validation_videos)]

In [26]:
# save dfs for datasets
training_df.to_pickle('./dfs/training.pkl')
validation_df.to_pickle('./dfs/validation.pkl')

In [27]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset('./dfs/' + x + '.pkl', data_transforms[x])  for x in ['training', 'validation']}
dataset_sizes = {x: len(datasets[x]) for x in ['training', 'validation']}

In [28]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['training', 'validation']}

## 3. Train

In [29]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['training', 'validation']:
            if phase == 'training':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in Bar(dataloaders[phase]):
                inputs = inputs.to(utils.device)
                labels = labels.to(utils.device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'training'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'training':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'training':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [30]:
model_ft = train_model(resnet18, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCHS)

Epoch 0/3
----------
164841/164841: [===============================>] - ETA 0.8sss
training Loss: 0.5654 Acc: 0.8119
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.5977 Acc: 0.8028

Epoch 1/3
----------
164841/164841: [===============================>] - ETA 0.6sss
training Loss: 0.2817 Acc: 0.9066
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.6362 Acc: 0.8035

Epoch 2/3
----------
164841/164841: [===============================>] - ETA 0.6sss
training Loss: 0.2054 Acc: 0.9336
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.6286 Acc: 0.8068

Epoch 3/3
----------
164841/164841: [===============================>] - ETA 0.6sss
training Loss: 0.1939 Acc: 0.9372
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.6406 Acc: 0.8064

Training complete in 64m 4s
Best val Acc: 0.806775
